In [3]:
# Offensive Content Detection with Contextual Understanding
# --------------------------------------------------------
# Enhanced with advanced suggestion generation using pre-trained models

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import pickle
import time
import requests
from io import StringIO
from tqdm.notebook import tqdm
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

print("Setting up the environment...")

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

2025-05-06 12:51:03.972014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746535864.234549      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746535864.312825      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Setting up the environment...
Using device: cuda


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
print("Loading dataset...")
file_path = "/kaggle/input/offensivetext/train.csv"
data = pd.read_csv(file_path)

print(f"Dataset loaded with shape: {data.shape}")
print("\nFirst few rows of the dataset:")
print(data.head())

# Display basic statistics
print("\nBasic statistics of the dataset:")
print(data.describe())

# Check for missing values
print("\nMissing values in each column:")
print(data.isnull().sum())

# Distribution of labels
print("\nDistribution of labels:")
for col in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    print(f"{col}: {data[col].value_counts()}")

Loading dataset...
Dataset loaded with shape: (159571, 8)

First few rows of the dataset:
                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  

Basic statistics of the dataset:
               toxic   severe_toxic      

In [5]:
# Create a binary toxicity label (any type of toxicity)
data['any_toxic'] = data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)
print("\nOverall toxicity distribution:")
print(data['any_toxic'].value_counts())
print(f"Percentage of toxic comments: {data['any_toxic'].mean() * 100:.2f}%")

# Preprocess text data
def preprocess_text(text):
    """Basic preprocessing for text data"""
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()
        # Remove URLs
        text = re.sub(r'https?://\S+|www\.\S+', '', text)
        # Remove HTML tags
        text = re.sub(r'<.*?>', '', text)
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Remove extra spaces
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ""

# Sample some comments to examine
print("\nSample non-toxic comments:")
non_toxic_samples = data[data['any_toxic'] == 0]['comment_text'].sample(5).tolist()
for i, comment in enumerate(non_toxic_samples):
    print(f"{i+1}. {comment[:100]}...")

print("\nSample toxic comments:")
toxic_samples = data[data['any_toxic'] == 1]['comment_text'].sample(5).tolist()
for i, comment in enumerate(toxic_samples):
    print(f"{i+1}. {comment[:100]}...")


Overall toxicity distribution:
any_toxic
0    143346
1     16225
Name: count, dtype: int64
Percentage of toxic comments: 10.17%

Sample non-toxic comments:
1. "

Oh, don't worry about me, Sandstein. I'm of no strong opinion as to what is ""well."" Editing Wik...
2. Are you trying to dispute that fact?...
3. SWOT analysis 

This source – Align Technology, Inc. SWOT Analysis. (2013). 1-8. – is used 11 times,...
4. cover 

so, do we want a current or older cover? i have the 57 paper edition i can scan. its actuall...
5. P.S. It's probably worth setting up a template to be used for all the periods, so they can easily be...

Sample toxic comments:
1. Sleazebag 

Jimbo, you are a sleazebag. Please go away....
2. The above user, who has failed to sign, is ignorant. The term 'nascent hydrogen' has a respectable s...
3. Don't worry, I'm in the mood for an edit war!  Let's fight this dirty communist!  Long live Romania!...
4. Dude 

Why do you fucking delete my shit when I didn't fucking do any

In [6]:
# Prepare the data for model training
print("\nPreparing data for model training...")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    data['comment_text'], 
    data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']], 
    test_size=0.2, 
    random_state=42
)

print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")

# Fine-tune a pre-trained model (RoBERTa)
MODEL_NAME = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Create a custom dataset
class ToxicityDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        item = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }
        
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels.iloc[idx].values, dtype=torch.float)
            
        return item

# Create the datasets
train_dataset = ToxicityDataset(X_train, y_train, tokenizer)
test_dataset = ToxicityDataset(X_test, y_test, tokenizer)

# Create dataloaders
BATCH_SIZE = 16
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)


Preparing data for model training...
Training set size: 127656
Testing set size: 31915


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=6,
    problem_type="multi_label_classification"
)
model.to(device)

# Training parameters
EPOCHS = 1
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_dataloader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# Training function
def train_epoch(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    
    progress_bar = tqdm(dataloader, desc="Training")
    for batch in progress_bar:
        # Move batch to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels
        )
        
        loss = outputs.loss
        total_loss += loss.item()
        
        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        # Update progress bar
        progress_bar.set_postfix({'loss': loss.item()})
    
    return total_loss / len(dataloader)

# Evaluation function
def evaluate(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)
            
            # Forward pass
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )
            
            logits = outputs.logits
            predictions.extend(torch.sigmoid(logits).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    predictions = np.array(predictions) >= 0.5
    true_labels = np.array(true_labels) >= 0.5
    
    return predictions, true_labels

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Train the model
print("\nTraining the model...")
train_losses = []

for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    start_time = time.time()
    
    # Train
    train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, device)
    train_losses.append(train_loss)
    
    # Evaluate
    print("Evaluating...")
    predictions, true_labels = evaluate(model, test_dataloader, device)
    
    # Calculate metrics
    accuracy = accuracy_score(true_labels.flatten(), predictions.flatten())
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels.flatten(), 
        predictions.flatten(), 
        average='binary'
    )
    
    # Print metrics
    print(f"Epoch {epoch+1} completed in {time.time() - start_time:.2f} seconds")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    
    # Print per-category metrics
    for i, category in enumerate(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']):
        cat_precision, cat_recall, cat_f1, _ = precision_recall_fscore_support(
            true_labels[:, i], 
            predictions[:, i], 
            average='binary'
        )
        print(f"{category}: Precision={cat_precision:.4f}, Recall={cat_recall:.4f}, F1={cat_f1:.4f}")


Training the model...
Epoch 1/1


Training:   0%|          | 0/7979 [00:00<?, ?it/s]

Evaluating...


Evaluating:   0%|          | 0/1995 [00:00<?, ?it/s]

Epoch 1 completed in 3549.74 seconds
Train Loss: 0.0480
Accuracy: 0.9847, Precision: 0.7971, Recall: 0.7870, F1: 0.7920
toxic: Precision=0.8463, Recall=0.8289, F1=0.8375
severe_toxic: Precision=0.5227, Recall=0.4299, F1=0.4718
obscene: Precision=0.8271, Recall=0.8536, F1=0.8402
threat: Precision=0.5077, Recall=0.4459, F1=0.4748
insult: Precision=0.7513, Recall=0.7900, F1=0.7702
identity_hate: Precision=0.6359, Recall=0.4218, F1=0.5072


In [9]:
# Save the model
print("\nSaving the model...")
model_path = "offensive_content_detection_model"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
print(f"Model saved to {model_path}")


Saving the model...
Model saved to offensive_content_detection_model


In [10]:
!zip -r output.zip /kaggle/working/
from IPython.display import FileLink

# Create a clickable download link
FileLink(r'output.zip')


  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)
  adding: kaggle/working/offensive_content_detection_model/ (stored 0%)
  adding: kaggle/working/offensive_content_detection_model/special_tokens_map.json (deflated 52%)
  adding: kaggle/working/offensive_content_detection_model/model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 8%)
  adding: kaggle/working/offensive_content_detection_model/merges.txt (deflated 53%)
  adding: kaggle/working/offensive_content_detection_model/vocab.json (deflated 59%)
  adding: kaggle/working/offensive_content_detection_model/tokenizer_config.json (deflated 75%)
  adding: kaggle/working/offensive_content_detection_model/tokenizer.json (deflated 82%)
  adding: kaggle/working/offensive_content_detection_model/config.json (deflated 55%)


/kaggle/working/output.zip

In [11]:
# Load T5 model for text rewriting
print("\nLoading T5 model for generating non-offensive alternatives...")
t5_model_name = "t5-base"
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_name)
t5_model.to(device)


Loading T5 model for generating non-offensive alternatives...


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [30]:
def analyze_context(text, toxicity_scores):
    """
    Completely redesigned context analysis with direct insult detection
    """
    # EDIT: First check for direct insults and attacks - these override other contexts
    direct_insult_patterns = [
        r'you(\'re| are) (a |an |such a )?(fucking |damn |stupid |idiotic |dumb |)?(idiot|stupid|dumb|moron|asshole|bitch|cunt|bastard|retard)',
        r'(fuck|screw|damn) you',
        r'i hate you',
        r'(go|get) (fuck|screw) yourself',
        r'(go|get) lost',
        r'(go|get) (the fuck)? out( of here)?',
        r'kill yourself',
        r'die',
        r'nobody (likes|cares about) you',
        r'you(\'re| are) (worthless|useless|pathetic)',
        r'you (suck|stink)',
        r'you(\'re| are) (a |an )?(fucking |damn |stupid |)?(joke|loser|failure)',
        r'shut (the fuck )?up',
        r'shut your (mouth|face)',
        r'(fuck|screw|damn) (off|you)',
        r'(you|your) (stupid|dumb|idiotic|moronic) (fuck|ass|bitch)',
        r'what (the fuck|the hell) (is|are) (you|this)'
    ]
    
    # Check for direct insults first - these override other contexts
    has_direct_insult = any(re.search(pattern, text.lower()) for pattern in direct_insult_patterns)
    
    # If it's a direct insult, return immediately with no context adjustment
    if has_direct_insult:
        return {
            'original_scores': toxicity_scores,
            'adjusted_scores': toxicity_scores,  # No adjustment for direct insults
            'context_adjustment': 0,
            'reasons': ["Direct insult or attack detected"],
            'context_type': {
                'direct_insult': True,
                'educational': False,
                'condemning': False,
                'quoting': False
            }
        }
    
    # If not a direct insult, proceed with normal context detection
    educational_keywords = [
        'research', 'study', 'learn', 'educate', 'inform', 'article', 'documentary',
        'paper', 'academic', 'lecture', 'class', 'teach', 'professor', 'student',
        'history', 'historical', 'analyze', 'analysis', 'examine', 'investigate',
        'discuss', 'discussed', 'explains', 'explained', 'report', 'reported'
    ]
    
    condemning_keywords = [
        'bad', 'wrong', 'terrible', 'shouldn\'t', 'should not', 'condemn', 'against', 'oppose',
        'harmful', 'unacceptable', 'inappropriate', 'immoral', 'evil', 'horrific', 'awful',
        'disgusting', 'reject', 'denounce', 'criticize', 'disagree', 'disapprove',
        'never', 'not', 'isn\'t', 'aren\'t', 'don\'t', 'doesn\'t', 'shouldn\'t', 'wouldn\'t',
        'must not', 'cannot', 'can\'t', 'stop', 'prevent', 'fight against', 'combat'
    ]
    
    # EDIT: More precise quoting indicators that require clear attribution
    quoting_indicators = [
        'said', 'quoted', 'according to', 'reported', 'states', 'stated',
        'mentioned', 'wrote', 'writes', 'claim', 'claims', 'argue', 'argues',
        'testimony', 'quote', 'citing', 'cited', 'reference', 'referenced'
    ]
    
    # EDIT: Require explicit quotation marks or attribution phrases for quoting context
    explicit_quoting_patterns = [
        r'"[^"]+"',                      # Double quotes
        r"'[^']+'",                      # Single quotes
        r"said[^.]*['\"]",               # Said followed by quote
        r"quoted[^.]*['\"]",             # Quoted followed by quote
        r"according to[^,.]+(,|\.)",     # According to someone
        r"reported that[^,.]+(,|\.)",    # Reported that
        r"testified that[^,.]+(,|\.)",   # Testified that
        r"statement[^,.]+(,|\.)",        # Statement from
        r"example of[^,.]+(,|\.)"        # Example of
    ]
    
    educational_patterns = [
        r'in (a|the) (documentary|article|paper|study|research|book|lecture|class)',
        r'(documentary|article|paper|study|research) (on|about|discussing)',
        r'(learn|learned|learning) about',
        r'(teach|taught|teaching) (about|how)',
        r'(discuss|discussed|discussing) (how|why|the)',
        r'(explain|explained|explaining) (how|why|the)',
        r'(study|studied|studying) (of|on|about)',
        r'(research|researched|researching) (on|about)'
    ]
    
    condemning_patterns = [
        r'(is|are|was|were) (bad|wrong|terrible|harmful|unacceptable)',
        r'should (not|never) be',
        r'(must|should|need to) (stop|prevent|avoid|condemn)',
        r'(don\'t|do not|doesn\'t|does not) (condone|accept|approve)',
        r'(fight|stand) against',
        r'(oppose|opposing|opposed to)',
        r'(reject|rejecting|rejected)',
        r'(criticize|criticizing|criticized)'
    ]
    
    # Check for educational context
    has_educational_context = any(keyword in text.lower() for keyword in educational_keywords)
    has_educational_pattern = any(re.search(pattern, text.lower()) for pattern in educational_patterns)
    
    # Check for condemning context
    has_condemning_context = any(keyword in text.lower() for keyword in condemning_keywords)
    has_condemning_pattern = any(re.search(pattern, text.lower()) for pattern in condemning_patterns)
    
    # EDIT: More stringent check for quoting context - require explicit quotation
    has_quoting_keyword = any(indicator in text.lower() for indicator in quoting_indicators)
    has_explicit_quote = any(re.search(pattern, text) for pattern in explicit_quoting_patterns)
    has_quoting_context = has_quoting_keyword and has_explicit_quote
    
    # Context scoring
    context_score = 0
    reasons = []
    
    # Educational context scoring
    if has_educational_context or has_educational_pattern:
        context_score -= 0.4
        reasons.append("Educational context detected")
    
    # Condemning context scoring
    if has_condemning_context or has_condemning_pattern:
        context_score -= 0.5
        reasons.append("Condemning harmful behavior")
    
    # Quoting context scoring - only if explicit quotation is present
    if has_quoting_context:
        context_score -= 0.4
        reasons.append("Quoting or reporting context")
    
    # Special case handling for common phrases that are incorrectly flagged
    special_phrases = {
        "rape is bad": -0.9,
        "rape is wrong": -0.9,
        "rape is terrible": -0.9,
        "sexual assault is": -0.9,
        "hate speech is": -0.9,
        "racism is": -0.9,
        "sexism is": -0.9,
        "homophobia is": -0.9,
        "transphobia is": -0.9,
        "discrimination is": -0.9,
        "harassment is": -0.9,
        "bullying is": -0.9
    }
    
    for phrase, adjustment in special_phrases.items():
        if phrase in text.lower():
            context_score += adjustment
            reasons.append(f"Special case: condemning phrase '{phrase}'")
    
    # Apply stronger adjustments for combined contexts
    if len(reasons) > 1:
        context_score -= 0.2
        reasons.append("Multiple contextual indicators strengthen non-offensive determination")
    
    # Ensure the adjustment doesn't go below a reasonable threshold
    context_score = max(context_score, -0.9)
    
    # Apply the context adjustment to each toxicity score
    adjusted_scores = {
        label: max(0, min(1, score + context_score)) 
        for label, score in toxicity_scores.items()
    }
    
    return {
        'original_scores': toxicity_scores,
        'adjusted_scores': adjusted_scores,
        'context_adjustment': context_score,
        'reasons': reasons,
        'context_type': {
            'direct_insult': False,
            'educational': has_educational_context or has_educational_pattern,
            'condemning': has_condemning_context or has_condemning_pattern,
            'quoting': has_quoting_context
        }
    }

In [31]:
# Function to identify offensive words in text
def identify_offensive_words(text, context_type=None):
    """
    Enhanced offensive word detection with better context handling
    """
    # EDIT: Check for direct insults first - these are always offensive regardless of context
    if context_type and context_type.get('direct_insult', False):
        # Extract the offensive words from the direct insult
        words = re.findall(r'\b\w+\b', text.lower())
        offensive_words = {}
        
        # Common offensive words in direct insults
        insult_words = {
            'fuck': 0.9, 'fucking': 0.9, 'fucked': 0.9, 'fucker': 0.9,
            'shit': 0.8, 'shitty': 0.8, 'bullshit': 0.7,
            'ass': 0.7, 'asshole': 0.9, 'jackass': 0.8,
            'bitch': 0.9, 'bitches': 0.9, 'bitching': 0.8,
            'damn': 0.6, 'goddamn': 0.7,
            'nigga': 1.0, 'nigger': 1.0, 'negro': 0.9,
            'whore': 0.9, 'slut': 0.9, 'hoe': 0.9,
            'cunt': 1.0, 'pussy': 0.8, 'dick': 0.8, 'cock': 0.8,
            'retard': 0.9, 'retarded': 0.9,
            'faggot': 1.0, 'fag': 0.9, 'homo': 0.8,
            'idiot': 0.7, 'stupid': 0.7, 'dumb': 0.7, 'moron': 0.8,
            'kill': 0.9, 'die': 0.9, 'suicide': 0.9,
            'hate': 0.8, 'loser': 0.7
        }
        
        for word in words:
            clean_word = ''.join(c for c in word if c.isalnum())
            if clean_word in insult_words:
                offensive_words[clean_word] = insult_words[clean_word]
        
        return offensive_words
    
    # Common offensive words dictionary with severity scores (0-1)
    offensive_words = {
        'fuck': 0.8, 'fucking': 0.8, 'fucked': 0.8, 'fucker': 0.9,
        'shit': 0.6, 'shitty': 0.7, 'bullshit': 0.6,
        'ass': 0.5, 'asshole': 0.8, 'jackass': 0.7,
        'bitch': 0.8, 'bitches': 0.8, 'bitching': 0.7,
        'damn': 0.4, 'goddamn': 0.6,
        'nigga': 0.9, 'nigger': 1.0, 'negro': 0.8,
        'whore': 0.9, 'slut': 0.9, 'hoe': 0.8,
        'cunt': 0.9, 'pussy': 0.8, 'dick': 0.7, 'cock': 0.7,
        'retard': 0.9, 'retarded': 0.9,
        'faggot': 1.0, 'fag': 0.9, 'homo': 0.7,
        'idiot': 0.5, 'stupid': 0.5, 'dumb': 0.5, 'moron': 0.6,
        'kill': 0.7, 'die': 0.6, 'suicide': 0.8,
        'hate': 0.6, 'loser': 0.5
    }
    
    # Words that should be treated differently in educational/condemning contexts
    contextual_words = {
        'rape': 0.9,
        'sexual assault': 0.9,
        'racist': 0.8,
        'sexist': 0.8,
        'homophobic': 0.8,
        'transphobic': 0.8,
        'nazi': 0.9,
        'terrorism': 0.8,
        'terrorist': 0.8,
        'genocide': 0.9,
        'slavery': 0.8,
        'holocaust': 0.9
    }
    
    found_words = {}
    words = re.findall(r'\b\w+\b', text.lower())
    
    # Check for multi-word offensive terms
    for term in contextual_words:
        if ' ' in term and term in text.lower():
            # Only add if not in an educational or condemning context
            if not context_type or (not context_type.get('educational', False) and 
                                   not context_type.get('condemning', False) and
                                   not context_type.get('quoting', False)):
                found_words[term] = contextual_words[term]
    
    # Check for single offensive words
    for word in words:
        # Clean word of punctuation for matching
        clean_word = ''.join(c for c in word if c.isalnum())
        
        if clean_word in offensive_words:
            # EDIT: For quoting context, only include if the word is extremely offensive
            if context_type and context_type.get('quoting', False):
                if offensive_words[clean_word] >= 0.9:  # Only include highly offensive words in quotes
                    found_words[clean_word] = offensive_words[clean_word]
            else:
                found_words[clean_word] = offensive_words[clean_word]
        elif clean_word in contextual_words:
            # Only add contextual words if not in an educational or condemning context
            if not context_type or (not context_type.get('educational', False) and 
                                   not context_type.get('condemning', False) and
                                   not context_type.get('quoting', False)):
                found_words[clean_word] = contextual_words[clean_word]
    
    return found_words

# Advanced function to generate non-offensive alternatives using T5
def predict_toxicity(text, model, tokenizer, device):
    """
    Completely redesigned toxicity prediction with better direct insult detection
    """
    # EDIT: First check for direct insults using pattern matching
    direct_insult_patterns = [
        r'you(\'re| are) (a |an |such a )?(fucking |damn |stupid |idiotic |dumb |)?(idiot|stupid|dumb|moron|asshole|bitch|cunt|bastard|retard)',
        r'(fuck|screw|damn) you',
        r'i hate you',
        r'(go|get) (fuck|screw) yourself',
        r'(go|get) lost',
        r'(go|get) (the fuck)? out( of here)?',
        r'kill yourself',
        r'die',
        r'nobody (likes|cares about) you',
        r'you(\'re| are) (worthless|useless|pathetic)',
        r'you (suck|stink)',
        r'you(\'re| are) (a |an )?(fucking |damn |stupid |)?(joke|loser|failure)',
        r'shut (the fuck )?up',
        r'shut your (mouth|face)',
        r'(fuck|screw|damn) (off|you)',
        r'(you|your) (stupid|dumb|idiotic|moronic) (fuck|ass|bitch)',
        r'what (the fuck|the hell) (is|are) (you|this)'
    ]
    
    # Check if text matches any direct insult pattern
    is_direct_insult = any(re.search(pattern, text.lower()) for pattern in direct_insult_patterns)
    
    # If it's a direct insult, we can skip the model prediction and mark as toxic
    if is_direct_insult:
        # Create dummy toxicity scores (high for all categories)
        toxicity_scores = {
            'toxic': 0.9,
            'severe_toxic': 0.7,
            'obscene': 0.8,
            'threat': 0.5,
            'insult': 0.9,
            'identity_hate': 0.5
        }
        
        # Get contextual analysis (which will detect the direct insult)
        contextual_analysis = analyze_context(text, toxicity_scores)
        
        # Identify offensive words
        offensive_words = identify_offensive_words(text, contextual_analysis['context_type'])
        
        # Generate alternative
        suggested_alternative = generate_alternative_with_t5(text, contextual_analysis, offensive_words)
        
        return {
            'text': text,
            'is_toxic': True,  # Direct insults are always toxic
            'toxicity_scores': toxicity_scores,
            'adjusted_scores': toxicity_scores,  # No adjustment for direct insults
            'contextual_analysis': contextual_analysis,
            'offensive_words': offensive_words,
            'suggested_alternative': suggested_alternative,
            'detection_method': 'pattern_matching'  # Indicate we used pattern matching
        }
    
    # If not a direct insult, proceed with model prediction
    # Tokenize text
    inputs = tokenizer(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    # Move inputs to device
    inputs = {key: val.to(device) for key, val in inputs.items()}
    
    # Get predictions
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.sigmoid(logits).cpu().numpy()[0]
    
    # Determine toxicity labels
    toxicity_labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    results = {label: float(prob) for label, prob in zip(toxicity_labels, probs)}
    
    # Analyze context
    contextual_analysis = analyze_context(text, results)
    
    # Get context type
    context_type = contextual_analysis['context_type']
    
    # Identify offensive words with context awareness
    offensive_words = identify_offensive_words(text, context_type)
    
    # EDIT: Improved toxicity determination logic
    adjusted_scores = contextual_analysis['adjusted_scores']
    
    # EDIT: Different thresholds based on context
    threshold = 0.5
    if context_type['educational']:
        threshold = 0.7  # Higher threshold for educational content
    elif context_type['condemning']:
        threshold = 0.8  # Even higher threshold for condemning content
    elif context_type['quoting']:
        threshold = 0.7  # Higher threshold for quoting content
    
    # EDIT: More nuanced toxicity determination
    is_toxic = False
    
    # Direct insults are always toxic
    if context_type['direct_insult']:
        is_toxic = True
    # Check if any adjusted score exceeds the threshold
    elif any(prob >= threshold for prob in adjusted_scores.values()):
        is_toxic = True
    # Check for offensive words, but respect context
    elif offensive_words:
        # In educational/condemning/quoting contexts, only consider highly offensive words
        if (context_type['educational'] or context_type['condemning'] or context_type['quoting']):
            is_toxic = any(score >= 0.9 for score in offensive_words.values())
        else:
            is_toxic = True
    
    # EDIT: Special case handling for common condemning phrases
    special_phrases = [
        "rape is bad", "rape is wrong", "rape is terrible", 
        "sexual assault is bad", "sexual assault is wrong",
        "racism is bad", "racism is wrong", "sexism is bad", "sexism is wrong",
        "hate speech is bad", "hate speech is wrong"
    ]
    
    if any(phrase in text.lower() for phrase in special_phrases):
        is_toxic = False
    
    # Generate alternative if toxic
    suggested_alternative = None
    if is_toxic:
        suggested_alternative = generate_alternative_with_t5(text, contextual_analysis, offensive_words)
    
    return {
        'text': text,
        'is_toxic': is_toxic,
        'toxicity_scores': results,
        'adjusted_scores': adjusted_scores,
        'contextual_analysis': contextual_analysis,
        'offensive_words': offensive_words,
        'suggested_alternative': suggested_alternative,
        'detection_method': 'model_prediction'  # Indicate we used model prediction
    }

In [32]:
#Add a new function to detect direct insults
def is_direct_insult(text):
    """
    Detect if text contains a direct insult using pattern matching
    """
    direct_insult_patterns = [
        r'you(\'re| are) (a |an |such a )?(fucking |damn |stupid |idiotic |dumb |)?(idiot|stupid|dumb|moron|asshole|bitch|cunt|bastard|retard)',
        r'(fuck|screw|damn) you',
        r'i hate you',
        r'(go|get) (fuck|screw) yourself',
        r'(go|get) lost',
        r'(go|get) (the fuck)? out( of here)?',
        r'kill yourself',
        r'die',
        r'nobody (likes|cares about) you',
        r'you(\'re| are) (worthless|useless|pathetic)',
        r'you (suck|stink)',
        r'you(\'re| are) (a |an )?(fucking |damn |stupid |)?(joke|loser|failure)',
        r'shut (the fuck )?up',
        r'shut your (mouth|face)',
        r'(fuck|screw|damn) (off|you)',
        r'(you|your) (stupid|dumb|idiotic|moronic) (fuck|ass|bitch)',
        r'what (the fuck|the hell) (is|are) (you|this)'
    ]
    
    return any(re.search(pattern, text.lower()) for pattern in direct_insult_patterns)

In [24]:
def simple_word_replacement(text, offensive_words):
    """
    Enhanced simple word replacement with better alternatives and context preservation
    """
    # EDIT: Expanded map of offensive words to non-offensive alternatives
    offensive_to_neutral = {
        'fuck': 'darn', 'fucking': 'really', 'fucked': 'messed up', 'fucker': 'person',
        'shit': 'stuff', 'shitty': 'poor', 'bullshit': 'nonsense',
        'ass': 'behind', 'asshole': 'jerk', 'jackass': 'fool',
        'bitch': 'difficult person', 'bitches': 'people', 'bitching': 'complaining',
        'damn': 'darn', 'goddamn': 'darn',
        'nigga': 'person', 'nigger': 'person', 'negro': 'person',
        'whore': 'person', 'slut': 'person', 'hoe': 'person',
        'cunt': 'person', 'pussy': 'coward', 'dick': 'jerk', 'cock': 'rooster',
        'retard': 'person', 'retarded': 'inappropriate',
        'faggot': 'person', 'fag': 'person', 'homo': 'person',
        'idiot': 'foolish person', 'stupid': 'unwise', 'dumb': 'uninformed', 'moron': 'foolish person',
        'kill': 'harm', 'die': 'leave', 'suicide': 'self-harm',
        'hate': 'dislike', 'loser': 'unfortunate person',
        # EDIT: Added more alternatives for contextual words
        'rape': 'sexual assault', 
        'nazi': 'extremist',
        'terrorist': 'extremist',
        'genocide': 'mass killing',
        'slavery': 'forced labor',
        'holocaust': 'genocide'
    }
    
    # EDIT: Improved tokenization to better preserve sentence structure
    tokens = []
    current_token = ""
    for char in text:
        if char.isalnum() or char == "'":
            current_token += char
        else:
            if current_token:
                tokens.append(current_token)
                current_token = ""
            if not char.isspace():
                tokens.append(char)
    if current_token:
        tokens.append(current_token)
    
    # EDIT: Improved replacement logic with phrase detection
    i = 0
    while i < len(tokens):
        # Check for multi-word phrases (up to 3 words)
        for phrase_len in range(3, 0, -1):
            if i + phrase_len <= len(tokens):
                # Create potential phrase
                potential_phrase = " ".join([t.lower() for t in tokens[i:i+phrase_len] 
                                           if t not in string.punctuation])
                
                if potential_phrase in offensive_to_neutral:
                    # Replace with alternative
                    replacement = offensive_to_neutral[potential_phrase]
                    
                    # Preserve capitalization of first token
                    if tokens[i][0].isupper() if tokens[i] and tokens[i][0].isalpha() else False:
                        replacement = replacement.capitalize()
                    
                    # Replace the first token with the alternative
                    tokens[i] = replacement
                    
                    # Remove the other tokens that were part of the phrase
                    for _ in range(phrase_len - 1):
                        if i + 1 < len(tokens):
                            tokens.pop(i + 1)
                    
                    break
        
        # Check single tokens
        token_lower = tokens[i].lower()
        if token_lower in offensive_to_neutral:
            # Preserve capitalization
            replacement = offensive_to_neutral[token_lower]
            if tokens[i].istitle():
                replacement = replacement.capitalize()
            elif tokens[i].isupper():
                replacement = replacement.upper()
            tokens[i] = replacement
        
        i += 1
    
    # Reconstruct the text with proper spacing
    result = ""
    for i, token in enumerate(tokens):
        if i > 0 and token in string.punctuation:
            result += token
        elif i > 0:
            result += " " + token
        else:
            result += token
    
    return result

In [25]:
# Function to predict toxicity with contextual understanding
def predict_toxicity(text, model, tokenizer, device):
    """
    Enhanced toxicity prediction with better threshold handling and context awareness
    """
    # Tokenize text
    inputs = tokenizer(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    # Move inputs to device
    inputs = {key: val.to(device) for key, val in inputs.items()}
    
    # Get predictions
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.sigmoid(logits).cpu().numpy()[0]
    
    # Determine toxicity labels
    toxicity_labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    results = {label: float(prob) for label, prob in zip(toxicity_labels, probs)}
    
    # Analyze context
    contextual_analysis = analyze_context(text, results)
    
    # EDIT: First determine context before identifying offensive words
    context_type = contextual_analysis['context_type']
    
    # Identify offensive words with context awareness
    offensive_words = identify_offensive_words(text, context_type)
    
    # EDIT: Improved toxicity determination logic
    adjusted_scores = contextual_analysis['adjusted_scores']
    
    # EDIT: Different thresholds based on context
    threshold = 0.5
    if context_type['educational']:
        threshold = 0.7  # Higher threshold for educational content
    elif context_type['condemning']:
        threshold = 0.8  # Even higher threshold for condemning content
    elif context_type['quoting']:
        threshold = 0.7  # Higher threshold for quoting content
    
    # EDIT: More nuanced toxicity determination
    is_toxic = False
    
    # Check if any adjusted score exceeds the threshold
    if any(prob >= threshold for prob in adjusted_scores.values()):
        is_toxic = True
    
    # Check for offensive words, but respect context
    if offensive_words:
        # In educational/condemning/quoting contexts, only consider highly offensive words
        if (context_type['educational'] or context_type['condemning'] or context_type['quoting']):
            is_toxic = any(score >= 0.9 for score in offensive_words.values())
        else:
            is_toxic = True
    
    # EDIT: Special case handling for common condemning phrases
    special_phrases = [
        "rape is bad", "rape is wrong", "rape is terrible", 
        "sexual assault is bad", "sexual assault is wrong",
        "racism is bad", "racism is wrong", "sexism is bad", "sexism is wrong",
        "hate speech is bad", "hate speech is wrong"
    ]
    
    if any(phrase in text.lower() for phrase in special_phrases):
        is_toxic = False
    
    # Generate alternative if toxic
    suggested_alternative = None
    if is_toxic:
        suggested_alternative = generate_alternative_with_t5(text, contextual_analysis, offensive_words)
    
    return {
        'text': text,
        'is_toxic': is_toxic,
        'toxicity_scores': results,
        'adjusted_scores': adjusted_scores,
        'contextual_analysis': contextual_analysis,
        'offensive_words': offensive_words,
        'suggested_alternative': suggested_alternative
    }

In [33]:
def interactive_prediction(model, tokenizer, device):
    """
    Enhanced interactive prediction with better output formatting
    """
    print("\n=== Offensive Content Detection Interactive Tool ===")
    print("Enter text to analyze (type 'exit' to quit):")
    
    while True:
        text = input("\nEnter text: ")
        if text.lower() == 'exit':
            break
        
        # EDIT: First check if it's a direct insult
        direct_insult = is_direct_insult(text)
        if direct_insult:
            print("\nDirect insult detected! Skipping model prediction.")
        
        # Get full prediction
        result = predict_toxicity(text, model, tokenizer, device)
        
        print("\n--- Analysis Results ---")
        print(f"Text: {result['text']}")
        print(f"Is offensive: {'Yes' if result['is_toxic'] else 'No'}")
        print(f"Detection method: {result['detection_method']}")
        
        print("\nToxicity scores:")
        for label, score in result['toxicity_scores'].items():
            print(f"  {label}: {score:.4f}")
        
        print("\nAdjusted scores (after contextual analysis):")
        for label, score in result['adjusted_scores'].items():
            print(f"  {label}: {score:.4f}")
        
        print("\nContextual analysis:")
        print(f"  Context adjustment: {result['contextual_analysis']['context_adjustment']}")
        if result['contextual_analysis']['reasons']:
            print("  Reasons:")
            for reason in result['contextual_analysis']['reasons']:
                print(f"    - {reason}")
        
        # EDIT: Improved offensive words display
        if result['offensive_words']:
            print("\nPotentially sensitive words detected:")
            for word, severity in result['offensive_words'].items():
                severity_level = "High" if severity >= 0.8 else "Medium" if severity >= 0.5 else "Low"
                print(f"  {word} (severity: {severity:.2f} - {severity_level})")
            
            # Explain why these words might not make the content offensive
            if not result['is_toxic'] and result['offensive_words']:
                print("\n  Note: These words were detected but determined to be non-offensive in this context.")
        
        # EDIT: Improved suggestion display
        if result['is_toxic']:
            print("\nSuggested non-offensive alternative:")
            if result['suggested_alternative']:
                print(f"  \"{result['suggested_alternative']}\"")
            else:
                # Better explanation when no suggestion is available
                if result['contextual_analysis']['context_type']['educational']:
                    print("  No suggestion provided as this appears to be educational content.")
                elif result['contextual_analysis']['context_type']['condemning']:
                    print("  No suggestion provided as this appears to be condemning harmful behavior.")
                elif result['contextual_analysis']['context_type']['quoting']:
                    print("  No suggestion provided as this appears to be quoting or reporting content.")
                else:
                    print("  No viable suggestion available for this content.")
        
        print("\n" + "-"*50)

In [34]:
# Run the interactive prediction tool
print("\nLoading the model for interactive prediction...")
interactive_prediction(model, tokenizer, device)

# Final model evaluation on test set
print("\nFinal model evaluation on test set:")
predictions, true_labels = evaluate(model, test_dataloader, device)

# Calculate overall metrics
accuracy = accuracy_score(true_labels.flatten(), predictions.flatten())
precision, recall, f1, _ = precision_recall_fscore_support(
    true_labels.flatten(), 
    predictions.flatten(), 
    average='binary'
)

print(f"Overall: Accuracy={accuracy:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, F1={f1:.4f}")

# Calculate per-category metrics
categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
for i, category in enumerate(categories):
    cat_precision, cat_recall, cat_f1, _ = precision_recall_fscore_support(
        true_labels[:, i], 
        predictions[:, i], 
        average='binary'
    )
    cat_accuracy = accuracy_score(true_labels[:, i], predictions[:, i])
    print(f"{category}: Accuracy={cat_accuracy:.4f}, Precision={cat_precision:.4f}, Recall={cat_recall:.4f}, F1={cat_f1:.4f}")


Loading the model for interactive prediction...

=== Offensive Content Detection Interactive Tool ===
Enter text to analyze (type 'exit' to quit):



Enter text:  fuck you bitch ass nigga



Direct insult detected! Skipping model prediction.

--- Analysis Results ---
Text: fuck you bitch ass nigga
Is offensive: Yes
Detection method: pattern_matching

Toxicity scores:
  toxic: 0.9000
  severe_toxic: 0.7000
  obscene: 0.8000
  threat: 0.5000
  insult: 0.9000
  identity_hate: 0.5000

Adjusted scores (after contextual analysis):
  toxic: 0.9000
  severe_toxic: 0.7000
  obscene: 0.8000
  threat: 0.5000
  insult: 0.9000
  identity_hate: 0.5000

Contextual analysis:
  Context adjustment: 0
  Reasons:
    - Direct insult or attack detected

Potentially sensitive words detected:
  fuck (severity: 0.90 - High)
  bitch (severity: 0.90 - High)
  ass (severity: 0.70 - Medium)
  nigga (severity: 1.00 - High)

Suggested non-offensive alternative:
  "darn you difficult person behind person"

--------------------------------------------------


KeyboardInterrupt: Interrupted by user

In [35]:
# Example of handling difficult cases (Advanced contextual analysis)
print("\nAdvanced contextual analysis examples:")

difficult_examples = [
    "Rape is terrible and should never be tolerated in society.",
    "The documentary discussed how hate speech affects minority communities.",
    "According to the article, racial slurs were common in that historical period.",
    "Research on offensive language shows interesting linguistic patterns.",
    "He quoted the offensive message that said 'fuck you' to demonstrate harassment.",
    "I hate you and wish you would die.",
    "You're such a fucking idiot.",
    "People from [demographic] are all [stereotype].",
    "Go kill yourself.",
    "What the fuck is this nigga talking about?"
]

for example in difficult_examples:
    result = predict_toxicity(example, model, tokenizer, device)
    
    print(f"\nText: {example}")
    print(f"Is offensive: {'Yes' if result['is_toxic'] else 'No'}")
    
    if result['is_toxic'] and result['suggested_alternative']:
        print(f"Suggested alternative: {result['suggested_alternative']}")
    elif result['is_toxic']:
        print("No viable suggestion available for this content.")
    
    print(f"Context analysis: {', '.join(result['contextual_analysis']['reasons']) if result['contextual_analysis']['reasons'] else 'No special context detected'}")
    print("-"*50)

print("\nThank you for using the Offensive Content Detection model!")


Advanced contextual analysis examples:

Text: Rape is terrible and should never be tolerated in society.
Is offensive: No
Context analysis: Condemning harmful behavior, Special case: condemning phrase 'rape is terrible', Multiple contextual indicators strengthen non-offensive determination
--------------------------------------------------

Text: The documentary discussed how hate speech affects minority communities.
Is offensive: No
Context analysis: Educational context detected
--------------------------------------------------

Text: According to the article, racial slurs were common in that historical period.
Is offensive: No
Context analysis: Educational context detected
--------------------------------------------------

Text: Research on offensive language shows interesting linguistic patterns.
Is offensive: No
Context analysis: Educational context detected
--------------------------------------------------

Text: He quoted the offensive message that said 'fuck you' to demonstr